In [26]:
# 导入pandas用于读取表格数据
import pandas as pd

# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import CountVectorizer

# 导入LogisticRegression回归模型
from sklearn.linear_model import LogisticRegression

# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

# 读取数据集
train = pd.read_csv('~/autodl-tmp/gitdir/datawhale/基于论文摘要的文本分类与关键词抽取挑战赛公开数据/train.csv')
train['title'] = train['title'].fillna('')
train['abstract'] = train['abstract'].fillna('')

test = pd.read_csv('~/autodl-tmp/gitdir/datawhale/基于论文摘要的文本分类与关键词抽取挑战赛测试集B/testB.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')

# 提取文本特征，生成训练集与测试集
train['text'] = train['title'].fillna('') + ' ' +  train['author'].fillna('') + ' ' + train['abstract'].fillna('')+ ' ' + train['Keywords'].fillna('')
test['text'] = test['title'].fillna('') + ' ' +  test['author'].fillna('') + ' ' + test['abstract'].fillna('')

# 添加停用词分析
stops =[i.strip() for i in open(r'/root/autodl-tmp/gitdir/datawhale/stop.txt',encoding='utf-8').readlines()]

#第一次用BOW算法，没有停止词，讯飞提交结果是0.671
#vector = CountVectorizer().fit(train['text'])
#第二次用停用词和TF_IDF算法，讯飞提交结果是0.759，增加了0.9个百分点
vector = TfidfVectorizer(stop_words=stops).fit(train['text'])
train_vector = vector.transform(train['text'])
test_vector = vector.transform(test['text'])

# 引入模型
model = LogisticRegression()

# 开始训练，这里可以考虑修改默认的batch_size与epoch来取得更好的效果
model.fit(train_vector, train['label'])

# 利用模型对测试集label标签进行预测
test['label'] = model.predict(test_vector)
test['Keywords'] = test['title'].fillna('')
# 生成任务一推测结果
test[['uuid', 'Keywords', 'label']].to_csv('submit_task1.csv', index=None)


/root/miniconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


In [1]:
!pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
!pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [3]:
#import 相关库
# 导入pandas用于读取表格数据
import pandas as pd

# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import CountVectorizer

# 导入LogisticRegression回归模型
from sklearn.linear_model import LogisticRegression

# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

特征提取是机器学习任务中的一个重要步骤。我们将训练数据的每一个维度称为一个特征，例如，如果我们想要基于二手车的品牌、价格、行驶里程数三个变量来预测二手车的价格，则品牌、价格、行驶里程数为该任务的三个特征。所谓特征提取，即从训练数据的特征集合中创建新的特征子集的过程。提取出来的特征子集特征数一般少于等于原特征数，但能够更好地表征训练数据的情况，使用提取出的特征子集能够取得更好的预测效果。对于 NLP、CV 任务，我们通常需要将文本、图像特征提取为计算机可以处理的数值向量特征。我们一般可以使用 sklearn 库中的 feature_extraction 包来实现文本与图片的特征提取。

TF-IDF(term frequency–inverse document frequency)是一种用于信息检索与数据挖掘的常用加权技术，其中，TF 指 term frequence，即词频，指某个词在文章中出现次数与文章总次数的比值；IDF 指 inverse document frequence，即逆文档频率，指包含某个词的文档数占语料库总文档数的比例。例如，假设语料库为 {"今天 天气 很好","今天 心情 很 不好", "明天 天气 不好"}，每一个句子为一个文档，则“今天”的 TF 和 IDF 分别为：

看文档-此处缺失
每个词最终的 IF-IDF 即为 TF 值乘以 IDF 值。计算出每个词的 TF-IDF 值后，使用 TF-IDF 计算得到的数值向量替代原文本即可实现基于 TF-IDF 的文本特征提取。

我们可以使用 sklearn.feature_extraction.text 中的 TfidfVectorizer 类来简单实现文档基于 TF-IDF 的特征提取：

In [7]:
# 首先导入该类
from sklearn.feature_extraction.text import TfidfVectorizer

# 假设我们已从本地读取数据为 DataFrame 类型，并已经过基本预处理，data 为已处理的 DataFrame 数据
# 实例化一个 TfidfVectorizer 对象，并使用 fit 方法来拟合数据
text=["hello world"]
vector = TfidfVectorizer().fit(text)

# 拟合之后，调用 transform 方法即可得到提取后的特征数据
train_vector = vector.transform(text)
train_vector.toarray()

array([[0.70710678, 0.70710678]])

基于 BOW

BOW（Bag of Words）是一种常用的文本表示方法，其基本思想是假定对于一个文本，忽略其词序和语法、句法，仅仅将其看做是一些词汇的集合，而文本中的每个词汇都是独立的。简单说就是讲每篇文档都看成一个袋子（因为里面装的都是词汇，所以称为词袋，Bag of words即因此而来），然后看这个袋子里装的都是些什么词汇，将其分类。具体而言，词袋模型表示一个文本，首先会维护一个词库，词库里维护了每一个词到一个数值向量的映射关系。例如，最简单的映射关系是独热编码，假设词库里一共有四个词，今天、天气、很、不好，那么独热编码会将四个词分别编码为：

今天——（1,0,0,0）

天气——（0,1,0,0）

很 ——（0,0,1,0）

不好——（0,0,0,1）

而使用词袋模型，就会将上述这句话编码为：

BOW(Sentence）=Embedding(今天)+Embedding(天气)+Embedding(很)+Embedding(不好)=(1,1,1,1）

In [8]:
# 首先导入该类
from sklearn.feature_extraction.text import CountVectorizer

# 假设我们已从本地读取数据为 DataFrame 类型，并已经过基本预处理，data 为已处理的 DataFrame 数据
# 实例化一个 CountVectorizer 对象，并使用 fit 方法来拟合数据
text=["hello"]
vector = CountVectorizer().fit(text)

# 拟合之后，调用 transform 方法即可得到提取后的特征数据
train_vector = vector.transform(text)
train_vector.toarray()

array([[1, 1]])

# 停用词

In [11]:
stops =[i.strip() for i in open(r'/root/autodl-tmp/gitdir/datawhale/stop.txt',encoding='utf-8').readlines()]

In [14]:
vector = CountVectorizer(stop_words=stops).fit(text)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [15]:
from sklearn.model_selection import train_test_split

# 该函数将会根据给定比例将数据集划分为训练集与验证集
trian_data, eval_data = train_test_split(data, test_size = 0.2)
# 参数 data 为总数据集，可以是 DataFrame 类型
# 参数 test_size 为划分验证集的占比，此处选择0.2，即划分20%样本作为验证集


NameError: name 'data' is not defined

# 选择机器学习模型

我们可以选择多种机器学习模型来拟合训练数据，不同的业务场景、不同的训练数据往往最优的模型也不同。常见的模型包括线性模型、逻辑回归、决策树、支持向量机、集成模型、神经网络等。想要深入学习各种机器学习模型的同学，推荐学习《西瓜书》或《统计学习方法》。

Sklearn 封装了多种机器学习模型，常见的模型都可以在 sklearn 中找到，sklearn 根据模型的类别组织在不同的包中，此处介绍几个常用包：

sklearn.linear_model：线性模型，如线性回归、逻辑回归、岭回归等
sklearn.tree：树模型，一般为决策树
sklearn.neighbors：最近邻模型，常见如 K 近邻算法
sklearn.svm：支持向量机
sklearn.ensemble：集成模型，如 AdaBoost、GBDT等
本案例中，我们使用简单但拟合效果较好的逻辑回归模型作为 Baseline 的模型。此处简要介绍其原理。

逻辑回归模型，即 Logistic Regression，实则为一个线性分类器，通过 Logistic 函数(或 Sigmoid 函数)，将数据特征映射到0～1区间的一个概率值（样本属于正例的可能性），通过与 0.5 的比对得出数据所属的分类。逻辑回归的数学表达式为：

In [16]:
# 引入模型
model = LogisticRegression()
# 可以在初始化时控制超参的取值，此处使用默认值，具体参数可以查阅官方文档

# 开始训练，这里可以考虑修改默认的batch_size与epoch来取得更好的效果
# 此处的 train_vector 是已经经过特征提取的训练数据
model.fit(train_vector, train['label'])

# 利用模型对测试集label标签进行预测，此处的 test_vector 同样是已经经过特征提取的测试数据
test['label'] = model.predict(test_vector)


NameError: name 'train' is not defined

In [18]:
import pandas as pd
train = pd.read_csv('/root/autodl-tmp/gitdir/datawhale/基于论文摘要的文本分类与关键词抽取挑战赛公开数据/train.csv')
train['title'] = train['title'].fillna('')
train['abstract'] = train['abstract'].fillna('')

test = pd.read_csv('/root/autodl-tmp/gitdir/datawhale/基于论文摘要的文本分类与关键词抽取挑战赛测试集B/testB.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')


In [20]:
print(train['title'].apply(len).describe())


count    6000.000000
mean       95.045333
std        33.454553
min         8.000000
25%        72.000000
50%        90.000000
75%       114.000000
max       285.000000
Name: title, dtype: float64


In [21]:
print(train["label"].value_counts())


label
0    3079
1    2921
Name: count, dtype: int64


In [22]:

# 提取文本特征，生成训练集与测试集
train['text'] = train['title'].fillna('') + ' ' +  train['author'].fillna('') + ' ' + train['abstract'].fillna('')+ ' ' + train['Keywords'].fillna('')
test['text'] = test['title'].fillna('') + ' ' +  test['author'].fillna('') + ' ' + test['abstract'].fillna('')


特征工程

In [23]:
#特征工程
vector = CountVectorizer().fit(train['text'])
train_vector = vector.transform(train['text'])
test_vector = vector.transform(test['text'])


In [24]:
# 模型训练
model = LogisticRegression()

# 开始训练，这里可以考虑修改默认的batch_size与epoch来取得更好的效果
model.fit(train_vector, train['label'])


LogisticRegression()

In [25]:
# 利用模型对测试集label标签进行预测
test['label'] = model.predict(test_vector)
test['Keywords'] = test['title'].fillna('')
# 生成任务一推测结果
test[['uuid', 'Keywords', 'label']].to_csv('submit_task1.csv', index=None)
